In [2]:
import os
from tensorflow.keras import models, layers, activations, optimizers, utils, losses, initializers, metrics, callbacks
import numpy as np 
import cv2 as cv 
import matplotlib as plt 

In [3]:
epochs = 100
batch_size = 64
patience = 10
learning_rate = 0.001
model_path = 'checkpoints/model.keras'

In [5]:
# Transformada de Fourier
def fft(img):
    img = np.fft.fft2(img)
    img = np.fft.fftshift(img)
    return img

# Inversa (retorna para imagem original)
def ifft(fimg):
    fimg = np.fft.ifftshift(fimg)
    fimg = np.fft.ifft2(fimg)
    return fimg

# Obtém a magnitude da imagem
def mag(img):
    absvalue = np.abs(img)
    magnitude = 20 * np.log(absvalue)
    return magnitude

# Normaliza a imagem entre 0 e 255
def norm(img):
    img = cv.normalize(
    img, None, 0, 255,
    cv.NORM_MINMAX
    )

# Melhor para ver imagens da transformada e imagens pequenas em geral.
def show(img):
    plt.imshow(img, cmap='gray')
    plt.show()
    return img


In [6]:
exists = os.path.exists(model_path)

model = models.load_model(model_path) \
    if exists \
        else models.Sequential([
            layers.Resizing(128, 128),
            layers.Rescaling(scale=1./127.5, offset=-1),
            layers.Conv2D(32, (3, 3),
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(32, (3, 3),
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.MaxPooling2D((2, 2)),
            layers.BatchNormalization(),
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(62,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dropout(0.5),
            layers.Dense(124,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dense(62,
                activation = 'sigmoid',
                kernel_initializer = initializers.RandomNormal()
            )
        ])

if exists:
    model.summary()
else:
    model.compile(
        optimizer = optimizers.Adam(
            learning_rate = learning_rate
        ),
        loss = losses.BinaryCrossentropy(),
        metrics = [ metrics.BinaryAccuracy() ]
    )

AttributeError: module 'keras.src.backend' has no attribute 'floatx'

In [ ]:
train = utils.image_dataset_from_directory(
    "img",
    validation_split= 0.2,
    subset= "training",
    seed= 123,
    shuffle= True,
    image_size= (128, 128),
    batch_size= batch_size
)

test = utils.image_dataset_from_directory(
    "img",
    validation_split= 0.2,
    subset= "validation",
    seed= 123,
    shuffle= True,
    image_size= (128, 128),
    batch_size= batch_size
)


In [ ]:
model.fit(train,
    epochs = epochs,
    validation_data = test,
    callbacks= [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks.ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)